# Exploring Perceptual Hashing for image-deduplication and as a low-hanging fruit for product-matching 
This notebook demonstrates the use-case of perceptual hashing applied on images for the following potential benefits:
- **Data de-duplication / cleaning**  
  Image dedup is impreative as a data preprocessing step for training image models 
- **Product-matching low hanging fruit**  
  Being able to identify duplicate images across merchants can serve as a low-hanging fruit for product-matching. Providing zero-error, free product-matches for a substanitally common occurence of finding visually identical product images across merchants.
- **Preload human-labelling tasks**  
  Using this list of free zero-error matches to preload human-in-the-loop tasks for product-matching, absolving the human annotator of matching obvious matches when the offer pair has same images.
- **Fair evaluation of the product-matching models**  
  Since an image-encoder model can be seen as just a glorified perceptual hashing function (even a randomly intialized model will match duplicate images), removing duplicate images in evaluation sets will help us in getting fairer performance metrics. 

## Introduction
Perceptual hashes tell whether two images look nearly identical. This is different from cryptographic hashing algorithms (like MD5, SHA-1) where tiny changes in the image give completely different hashes. In image fingerprinting, we actually want our similar inputs to have similar output hashes as well.

For a much better explaination check [this blog](https://www.hackerfactor.com/blog/index.php?/archives/432-Looks-Like-It.html).


## Problem Statement
The current data in our `offers_en` table has duplicates of the following kinds:
- different offer id and/or different a3_prod_id; but same title, same color label.
- different offer id and/or different a3_prod_id; slightly different title and color label but same offer.
- offer with identical offer images for its gender variants

Perceptual hashing is a great way to deal with this data cleaning problem but for its adaptiblity in our use-case there are certain challanges:
- Phashing also hashes nearly identical images to the same hash output. This will lead to error-propogation and will cause variants of the offers to be matched together. This is usually a desired outcome of Phashing but in our case we do not want near identical image hashing.
- Phashing in general is performed on grayscale images. This makes the Phashing algorithm invariant to color filters applied on images. This again is a desired behaviour of a Phashing algo but in our use-case we do not want indiffernce to colors since that would lead to color-variants being hashed together.

## Solution Statement
In order for the Phashing to work for our use-case we need to modify the original phash algo for the following outcomes:
- **Fast and cheap**  
  Popular phashing algos are all compute cheap and suitably belongs to lambda function.
- **Should be robust against change in image resolution, aspect ratio, JPEG compression.**  
  This is a pre-fulfilled expectation with phashing as Phashing is robust to these kind of perturbations.
- **Should not hash near-identical images as duplicates.**  
  This might be achieved by increasing the "resolution" of the hashing function.
- **Should not be indifferent to color profiles of the same image.**  
  This might be achieved by modifying repeating the hashing logic over the 3 channels and appending the 3 hashes.

In [71]:
import os
import random
import glob
from collections import defaultdict
import pickle
import uuid

import numpy as np
import pandas as pd
import ipyplot as iplt

data_dir = "/data/"

# Fetch data

In [72]:
df = pd.read_parquet("s3://aisle3-ml-datasets/product-matching/aisle3/clean_poses.parquet")
df

,id,variant_id,title,merchant,brand,gender,color,imid,image_url,image,pose
0,allsole.10491511,allsole.10491633,Vans Authentic Canvas Trainers,allsole,vans,unisex,Black,26789,https://aisle-3-image-final.s3.eu-west-2.amazo...,s3://aisle-3-image-final/images/original/allso...,side_shot
1,allsole.10491511,allsole.10491633,Vans Authentic Canvas Trainers,allsole,vans,unisex,Black,26790,https://aisle-3-image-final.s3.eu-west-2.amazo...,s3://aisle-3-image-final/images/original/allso...,side_shot
2,allsole.10491511,allsole.10491633,Vans Authentic Canvas Trainers,allsole,vans,unisex,Black,26791,https://aisle-3-image-final.s3.eu-west-2.amazo...,s3://aisle-3-image-final/images/original/allso...,side_shot
3,allsole.10491511,allsole.10491633,Vans Authentic Canvas Trainers,allsole,vans,unisex,Black,26792,https://aisle-3-image-final.s3.eu-west-2.amazo...,s3://aisle-3-image-final/images/original/allso...,upper_shot
4,allsole.10491511,allsole.10491633,Vans Authentic Canvas Trainers,allsole,vans,unisex,Black,26793,https://aisle-3-image-final.s3.eu-west-2.amazo...,s3://aisle-3-image-final/images/original/allso...,partial_shot
...,...,...,...,...,...,...,...,...,...,...,...
202555,ssense.221903M237021,ssense.221903M237021,Coach 1941 Black & Off-White Logo Slide Sandals,ssense,coach,men,Chalk black,95785,https://aisle-3-image-final.s3.eu-west-2.amazo...,s3://aisle-3-image-final/images/original/ssens...,partial_shot
202556,ssense.221903M237021,ssense.221903M237021,Coach 1941 Black & Off-White Logo Slide Sandals,ssense,coach,men,Chalk black,95786,https://aisle-3-image-final.s3.eu-west-2.amazo...,s3://aisle-3-image-final/images/original/ssens...,pair_shot
202557,ssense.221903M237021,ssense.221903M237021,Coach 1941 Black & Off-White Logo Slide Sandals,ssense,coach,men,Chalk black,95787,https://aisle-3-image-final.s3.eu-west-2.amazo...,s3://aisle-3-image-final/images/original/ssens...,side_shot
202558,ssense.221903M237021,ssense.221903M237021,Coach 1941 Black & Off-White Logo Slide Sandals,ssense,coach,men,Chalk black,95788,https://aisle-3-image-final.s3.eu-west-2.amazo...,s3://aisle-3-image-final/images/original/ssens...,pair_shot


In [73]:
df.image = df.image.str.replace("s3://", "")
df.image

0         aisle-3-image-final/images/original/allsole/al...
1         aisle-3-image-final/images/original/allsole/al...
2         aisle-3-image-final/images/original/allsole/al...
3         aisle-3-image-final/images/original/allsole/al...
4         aisle-3-image-final/images/original/allsole/al...
                                ...                        
202555    aisle-3-image-final/images/original/ssense/sse...
202556    aisle-3-image-final/images/original/ssense/sse...
202557    aisle-3-image-final/images/original/ssense/sse...
202558    aisle-3-image-final/images/original/ssense/sse...
202559    aisle-3-image-final/images/original/ssense/sse...
Name: image, Length: 202560, dtype: object

In [74]:
(data_dir + df.image).apply(os.path.isfile).value_counts()

True    202560
Name: image, dtype: int64

# Implementation
From my experiments I found the following modifications to the plain **Phash** algorithm works well:
- Increase the `hash-size` (size of the top left square of DCT frequencies taken for hash) to introduce a greater avalanche factor into the hashing function.
- Repeate the calculation 3 times on the 3 color channels to extend Phash to be color sensitive.

In [75]:
import hashlib

import cv2
from PIL import Image
import numpy as np
import scipy.fftpack


def binary_to_hex(arr):
    bit_string = "".join(str(b) for b in 1 * arr.flatten())
    width = int(np.ceil(len(bit_string) / 4))
    return "{:0>{width}x}".format(int(bit_string, 2), width=width)


def phash_image(image, hash_size=10, highfreq_factor=6):
    img_size = hash_size * highfreq_factor
    # img = cv2.imread(image)
    img = Image.open(image).convert("RGB")
    # img = cv2.resize(img, (img_size, img_size), interpolation=cv2.INTER_AREA)
    img = img.resize((img_size, img_size), Image.ANTIALIAS)
    img = np.asarray(img)
    channel_hashes = list()
    channels = [img[:, :, i] for i in range(img.shape[2])]
    for channel in channels:
        dct = scipy.fftpack.dct(scipy.fftpack.dct(channel, axis=0), axis=1)
        dctlowfreq = dct[:hash_size, :hash_size]
        med = np.median(dctlowfreq)
        diff = dctlowfreq > med
        channel_hashes.append(binary_to_hex(diff))
    return "-".join(channel_hashes)


def md5_hash(file):
    hash_md5 = hashlib.md5()
    with open(file, "rb") as f:
        for chunk in iter(lambda: f.read(4096), b""):
            hash_md5.update(chunk)
    return hash_md5.hexdigest()

### Perform hashing
- phash image hash
- md5 file hash

In [76]:
from tqdm.contrib.concurrent import process_map, thread_map

df["phash"] = pd.Series(
    process_map(
        phash_image,
        (data_dir + df.image).values,
        max_workers=20,
        chunksize=100,
    )
)

df["md5"] =  pd.Series(
    thread_map(
        md5_hash,
        (data_dir + df.image).values,
        max_workers=10,
        chunksize=100,
    )
)

  0%|          | 0/202560 [00:00<?, ?it/s]

/home/sour4bh/.local/lib/python3.8/site-packages/PIL/Image.py:945: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/home/sour4bh/.local/lib/python3.8/site-packages/PIL/Image.py:945: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/home/sour4bh/.local/lib/python3.8/site-packages/PIL/Image.py:945: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/home/sour4bh/.local/lib/python3.8/site-packages/PIL/Image.py:945: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/home/sour4bh/.local/lib/python3.8/site-packages/PIL/Image.py:945: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


  0%|          | 0/202560 [00:00<?, ?it/s]

# Duplicates

### phash : How many duplicates are there? 
**distribution of image duplicates:**

In [77]:
df.phash.value_counts().value_counts().sort_index()

1    198973
2      1737
3        27
4         8
Name: phash, dtype: int64

### md5 : How many duplicates are there?
This shows that we have large part of the duplicates simply as actual image file duplicates. This is not to be understood as a very low recall of the Phashing function (technically we actually don't want recall).  
This is understandable given the main motive of this exercise -- removal of same merchant image duplicates.  

We will remove same md5 clusters later to isolate pHash clusters for an unbiased look.

**distribution of file duplicates:**

In [78]:
df.md5.value_counts().value_counts().sort_index()

1    200903
2       781
3        21
4         8
Name: md5, dtype: int64

## Let's visualize Image clusters with same pHashes

In [79]:
clusters = df.phash.value_counts().sort_values(ascending=False)
clusters

beb053e8e8c14fac10deda287-beb053e8e8c14fac10dbda287-beb053e8e8c14fac10dfda285    4
afe35d015a911ca8f8c54e8d7-afe35d015a911ca8f8c54e8d7-afe35d017a911ca8f8c54e8d5    4
e953996ac26953d96cc23133d-e9719968c26973d96cc23133d-e971d968c26973d96cc26131d    4
b83e0dfb1f674dc18f006c0e3-b83e0dfb1f670dc18f026c0e3-b83e0dfb1f674dc18f006c0e3    4
b913ec6cf193b1d2c64667231-f916fce4f093b8d0c64663239-f916dce4b893a8f0c74633239    4
                                                                                ..
aba0143433c8c9faf4d5af0f4-aba014303bc8c9faf4b5af0f4-aba014303bc8c9faf4b5af0f4    1
bfae48c702eb3c8c470be18da-bf2e48c703eb3c8c470be18da-bfae48c743eb388c470be18d8    1
baef34825493a5c3e25d46f0c-facf34824493a5c3e2dd46f18-eecf3c824493a5c3ea9c46f18    1
f671293b9819b191db84aef90-f671213f9019b191db84aef94-f6712137901bb191db843ef94    1
bff84798078047685ce25bb2b-bff84798078447685ca25bb2b-bff84798078447685ca25bb2b    1
Name: phash, Length: 200745, dtype: int64

### Top 20 clusters by size:

In [81]:
phash_clusters = clusters.index.tolist()
for cluster in phash_clusters[:20]:
    iplt.plot_images(
        df[df.phash == cluster].image_url.values,
        show_url=False,
        img_width=150
    )

## Do some in-notebook annotation:
We can see that some of these clusters are due to dirty non-product images so that the pose estimation missed out on, so let's annotate them out for now!  

In [82]:
from pigeon import annotate
from IPython.display import display, Image

df.set_index("phash", drop=False, inplace=True)


def display_fn(phash):
    frame = df[df.phash == phash]
    iplt.plot_images(
        frame.image_url.values,
        labels=frame.id.values,
        custom_texts=(
            "<HR>"
            + frame[
                [
                    "pose",
                    "title",
                ]
            ].apply("<HR>".join, axis=1)
            + "<HR>"
            + frame[["brand", "merchant", "color"]].apply(" - ".join, axis=1)
        ).values,
        show_url=False,
        img_width=150,
    )


annotations = annotate(
    phash_clusters, options=["Bad Image"], include_skip=True, display_fn=display_fn
)


HTML(value='0 examples annotated, 200746 examples left')

Output()

### Dirty / non-descriptive images:
I collected the following bad images by trimming the big clusters from the top, after a point these kind of images were hard to come by. We don't really have a lot of bad images (just a lot of duplicates of a few bad ones).  
**Pose estimation** can help here so that we don't use broken images, box shots, sole angles, charts/guides/anything that doesn't show the product -- as these images could be common to multiple non-identical offers.

In [83]:
bad_images = [x[0] for x in annotations]
iplt.plot_images(
    df[df.phash.isin(bad_images)].drop_duplicates("phash").image_url,
    show_url=False,
)
df = df.loc[~df.phash.isin(bad_images)].reset_index(drop=True)

## Graph connected components

If two offers **(either from the same merchant or from different merchants)** have any image which is duplicated in both these offers, then that counts as an "edge" connecting said two offers.  
Once the Graph is built we look at the connected components to find product clusters of offers.

In [84]:
import networkx as nx

G = nx.Graph()
for phash, phash_group in df.groupby("phash"):
    nodes = phash_group.id.unique()
    if len(nodes) > 1:
        for i in range(len(nodes) - 1):
            G.add_edge(nodes[i], nodes[i + 1])
print(
    f"Number of nodes : {G.number_of_nodes()}; Number of edges : {G.number_of_edges()}"
)
clusters = sorted(list(nx.connected_components(G)), key=len, reverse=True)
print(f"Number of clusters : {len(clusters)}")

Number of nodes : 1091; Number of edges : 550
Number of clusters : 541


### Cluster size distribution

In [85]:
pd.Series(clusters).apply(len).value_counts()

2    533
3      7
4      1
dtype: int64

## Visualize product clusters 

In [ ]:
df.set_index("id", drop=False, inplace=True)

def visualize_cluster(df):
    df = df.copy().reset_index(drop=True)
    iplt.plot_images(
            df.image_url,
            labels=df.id,
            custom_texts="<HR>"
            + "<b>"
            + df["title"]
            + "</b>"
            + "<HR>"
            + df[["merchant", "brand", "gender", "color"]].apply(" - ".join, axis=1)
            + "<HR>"
            + df.pose,
            img_width=200,
            show_url=False,
            max_images=len(df),
        )

for cluster in clusters[:100]:
    visualize_cluster(df.loc[cluster])

# Randomly sample 100 more clusters

In [ ]:
for cluster in random.choices(clusters, k=50):
    visualize_cluster(df.loc[cluster])

## What merchants are matched by phashing?

In [88]:
cluster_merchants = list()
for cluster in clusters:
    merchants = df.loc[cluster].merchant.unique()
    if len(merchants) > 1:
        cluster_merchants.append([m.upper() for m in merchants])
    else:
        cluster_merchants.append(merchants.tolist() * 2)

pd.Series(list(map(" <-> ".join, np.sort(cluster_merchants, axis=1)))).value_counts()

ALLSOLE <-> COGGLES                246
asos <-> asos                       97
sportsdirect <-> sportsdirect       58
schuh <-> schuh                     49
NIKE <-> SPORTSDIRECT               39
end clothing <-> end clothing       12
ssense <-> ssense                   11
spartoo.co.uk <-> spartoo.co.uk      7
new balance <-> new balance          7
FOOT LOCKER <-> NEW BALANCE          3
camper <-> camper                    3
clarks <-> clarks                    3
coggles <-> coggles                  2
nike <-> nike                        2
jd sports <-> jd sports              1
footasylum <-> footasylum            1
dtype: int64

# Conclusion
- For this technique to be used to get free product-matching matches we need to first pass these images through an outlier check and also discard singals from non-descriptive images since they are prone to give False Positives.

- Finding connected components retaining only the stronger connections does avert the absence of a pose estimation pass, and this can easily be used as "free" ground-truth product-matching. 

- For the purpose of image data cleaning / processing we can easily implement this technique. 

- This technique also bootstraps some text product-matching data as we collect matching product titles by matching solely using images.


In [89]:
pd.Series(clusters).to_json("s3://aisle3-ml-datasets/phash_clusters.json", orient="records", indent=1)

In [90]:
df["label"] = df.id.astype("category").cat.codes
for cluster in clusters:
    df.loc[cluster, "label"] = uuid.uuid4()

In [91]:
df = df.drop_duplicates("phash", keep="first").reset_index(drop=True)
df.label = df.label.astype("category").cat.codes
df

,id,variant_id,title,merchant,brand,gender,color,imid,image_url,image,pose,phash,md5,label
0,allsole.10491511,allsole.10491633,Vans Authentic Canvas Trainers,allsole,vans,unisex,Black,26789,https://aisle-3-image-final.s3.eu-west-2.amazo...,aisle-3-image-final/images/original/allsole/al...,side_shot,aff00c80ffd01b337b006ccde-aff40c80ffd01a337b00...,22e646989821796152307bef73fbc568,0
1,allsole.10491511,allsole.10491633,Vans Authentic Canvas Trainers,allsole,vans,unisex,Black,26790,https://aisle-3-image-final.s3.eu-west-2.amazo...,aisle-3-image-final/images/original/allsole/al...,side_shot,fea15c98da845223fb91e446e-f6a15c98fa845223fb99...,2875924f6d79dd99bc669cdbe84d82c7,0
2,allsole.10491511,allsole.10491633,Vans Authentic Canvas Trainers,allsole,vans,unisex,Black,26791,https://aisle-3-image-final.s3.eu-west-2.amazo...,aisle-3-image-final/images/original/allsole/al...,side_shot,aba51ec0e7931a89d54a6fa15-aba51ec0e7931a89d54a...,5da5243580c580f0cce4bfd54f6dd35a,0
3,allsole.10491511,allsole.10491633,Vans Authentic Canvas Trainers,allsole,vans,unisex,Black,26792,https://aisle-3-image-final.s3.eu-west-2.amazo...,aisle-3-image-final/images/original/allsole/al...,upper_shot,e7a66ce0e29c599987399a499-e7a66c60e69c59998739...,f28aae91e89aa1de723d993580253b61,0
4,allsole.10491511,allsole.10491633,Vans Authentic Canvas Trainers,allsole,vans,unisex,Black,26793,https://aisle-3-image-final.s3.eu-west-2.amazo...,aisle-3-image-final/images/original/allsole/al...,partial_shot,bc8d2d0f0f4b73c9d128d24e5-bc8d0c0f2f4b71c9d1a8...,393a50c1c362a7189130a944ee897f73,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200738,ssense.221903M237021,ssense.221903M237021,Coach 1941 Black & Off-White Logo Slide Sandals,ssense,coach,men,Chalk black,95785,https://aisle-3-image-final.s3.eu-west-2.amazo...,aisle-3-image-final/images/original/ssense/sse...,partial_shot,baf905ac67e514ba9542c00ff-baf905ac67e514ba9542...,fd39a5006695521400f9849a3aeb67de,45263
200739,ssense.221903M237021,ssense.221903M237021,Coach 1941 Black & Off-White Logo Slide Sandals,ssense,coach,men,Chalk black,95786,https://aisle-3-image-final.s3.eu-west-2.amazo...,aisle-3-image-final/images/original/ssense/sse...,pair_shot,ffe007805ec5ca7b805ec45e6-ffe007805ec5ca7b805e...,7cf2113b8e0c1710499bddd7df584fe3,45263
200740,ssense.221903M237021,ssense.221903M237021,Coach 1941 Black & Off-White Logo Slide Sandals,ssense,coach,men,Chalk black,95787,https://aisle-3-image-final.s3.eu-west-2.amazo...,aisle-3-image-final/images/original/ssense/sse...,side_shot,eeac14fb32b053ef8217d21a2-eeac54fb32b053ef8017...,f8553fbe9f45b50038b60cd6583af1a3,45263
200741,ssense.221903M237021,ssense.221903M237021,Coach 1941 Black & Off-White Logo Slide Sandals,ssense,coach,men,Chalk black,95788,https://aisle-3-image-final.s3.eu-west-2.amazo...,aisle-3-image-final/images/original/ssense/sse...,pair_shot,ffaa46cb15804f2ccb1dc20de-ffaa46ca15804f2ccb1d...,7ad47c5d9bacfa208d68545964717006,45263


In [92]:
df.image = "s3://" + df.image
# df.drop(columns=["phash", "md5"], inplace=True)
df.to_parquet("s3://aisle3-ml-datasets/product-matching/aisle3/deduped.parquet")